In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import os
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

from itertools import chain
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
class ReadInData(object):
    def __init__(self, path_to_data, properties_file, train_file):
        # Input path
        self.path_to_data = path_to_data
        
        # Data type definition
        self.dtype_dict = {
            'parcelid':                        np.int32,
            'airconditioningtypeid':           np.float64,
            'architecturalstyletypeid':        np.float64,
            'basementsqft':                    np.float64,
            'bathroomcnt':                     np.float64,
            'bedroomcnt':                      np.float64,
            'buildingclasstypeid':             np.float64,
            'buildingqualitytypeid':           np.float64,
            'calculatedbathnbr':               np.float64,
            'decktypeid':                      np.float64,
            'finishedfloor1squarefeet':        np.float64,
            'calculatedfinishedsquarefeet':    np.float64,
            'finishedsquarefeet12':            np.float64,
            'finishedsquarefeet13':            np.float64,
            'finishedsquarefeet15':            np.float64,
            'finishedsquarefeet50':            np.float64,
            'finishedsquarefeet6':             np.float64,
            'fips':                            np.float64,
            'fireplacecnt':                    np.float64,
            'fullbathcnt':                     np.float64,
            'garagecarcnt':                    np.float64,
            'garagetotalsqft':                 np.float64,
            'hashottuborspa':                  np.bool_,
            'heatingorsystemtypeid':           np.float64,
            'latitude':                        np.float64,
            'longitude':                       np.float64,
            'lotsizesquarefeet':               np.float64,
            'poolcnt':                         np.float64,
            'poolsizesum':                     np.float64,
            'pooltypeid10':                    np.float64,
            'pooltypeid2':                     np.float64,
            'pooltypeid7':                     np.float64,
            'propertycountylandusecode':       str,
            'propertylandusetypeid':           np.float64,
            'propertyzoningdesc':              str,
            'rawcensustractandblock':          np.float64,
            'regionidcity':                    np.float64,
            'regionidcounty':                  np.float64,
            'regionidneighborhood':            np.float64,
            'regionidzip':                     np.float64,
            'roomcnt':                         np.float64,
            'storytypeid':                     np.float64,
            'threequarterbathnbr':             np.float64,
            'typeconstructiontypeid':          np.float64,
            'unitcnt':                         np.float64,
            'yardbuildingsqft17':              np.float64,
            'yardbuildingsqft26':              np.float64,
            'yearbuilt':                       np.float64,
            'numberofstories':                 np.float64,
            'fireplaceflag':                   np.bool_,
            'structuretaxvaluedollarcnt':      np.float64,
            'taxvaluedollarcnt':               np.float64,
            'assessmentyear':                  np.float64,
            'landtaxvaluedollarcnt':           np.float64,
            'taxamount':                       np.float64,
            'taxdelinquencyflag':              str,
            'taxdelinquencyyear':              np.float64,
            'censustractandblock':             np.float64,
        }
        
        # Categorical columns with many unique values
        self.mean_cols = ['propertyzoningdesc', 
             'propertycountylandusecode', 
             'rawcensustractandblock', 
             'regionidcity',
             'regionidneighborhood',
             'regionidzip',
             'censustractandblock',
             'assessmentyear'
            ]
        
        for column in self.mean_cols:
            self.dtype_dict[column] = 'category'
        
        # File names
        self.properties_file = properties_file
        self.train_file = train_file
        
        # Imported datasets
        self.properties = pd.DataFrame()
        self.train_driver = pd.DataFrame()
        
    def import_data(self):
        self.train_driver = pd.read_csv(os.path.join(self.path_to_data, self.train_file), 
                                        parse_dates = ["transactiondate"])
        
        self.properties = pd.read_csv(os.path.join(self.path_to_data, self.properties_file),
                                      dtype=self.dtype_dict)
        self.properties = self.properties[self.properties.latitude.notnull()]
        #Need to convert categorical variables to categorical data types    

In [3]:
class CreateDatasets(ReadInData):
    def __init__(self, **kwargs):
        super(CreateDatasets, self).__init__(**kwargs)
        
        # Compiled datasets
        self.y_train = pd.DataFrame()
        self.scoring_driver = pd.DataFrame()
        self.Xs = {'x_train': pd.DataFrame(), 'x_scoring': pd.DataFrame()}

        # Special columns to treat differently
        self.drop_indices = ['parcelid']
        self.y_col = 'logerror'

        self.mean_col_dfs_ = {}
    
    def calculate_categorical_means(self, df, cat_column):
        # Calculate mean logerrors for categorical variables
        categorical_means = (df[['logerror', cat_column]]
                                          .groupby(cat_column)
                                          .agg({'logerror': 'mean'})
                                          .rename(columns={'logerror': cat_column+'_mean'}))
        categorical_means.reset_index(inplace=True) 
        categorical_means[cat_column+'_mean'].fillna(0, inplace=True)
        return categorical_means
    
    def create_scoring_driver(self, properties_df):
        # Start by identifying unique parcels
        parcels = properties_df.parcelid.unique()
        
        # Adding to a dataframe with a dummy variable that will allow a cross join in the next step
        parcels_df = pd.DataFrame({'parcelid': parcels, 'dummy': np.zeros(len(parcels))})

        #Create a row for each parcel for each month to be scored
        trxn_months = pd.DataFrame({'transactiondate': ['2016/10/01', '2016/11/01', '2016/12/01', '2017/10/01', '2017/11/01', '2017/12/01'],
                            'logerror': np.zeros(6),
                            'dummy': np.zeros(6)})

        trxn_months.transactiondate = pd.to_datetime(trxn_months.transactiondate)

        # Finally create the driver for scoring
        return pd.merge(parcels_df, trxn_months, how='outer', on='dummy').drop('dummy', axis=1)
    
    def run(self):
        self.import_data()
            
        train = pd.merge(self.properties, self.train_driver, how='left', on='parcelid')
        self.y_train = train[self.y_col]
        self.Xs['x_train'] = train.drop(self.drop_indices+[self.y_col], axis=1)
        
        self.scoring_driver = self.create_scoring_driver(self.properties)
        self.scoring = pd.merge(self.scoring_driver, self.properties, how='left', on='parcelid')
        self.Xs['x_scoring'] = self.scoring.drop(self.drop_indices, axis=1)
        
        # Calculate categorical means
        for column in self.mean_cols:
            self.mean_col_dfs_[column] = self.calculate_categorical_means(df=train, cat_column=column)
        
        self.Xs['x_train'] = self.Xs['x_train'][self.Xs['x_train']['transactiondate'].notnull()]
        self.y_train = self.y_train[self.y_train.notnull()]
        
        # Perform transforms that won't work in the pipeline:
        # - Drop transactiondate
        # - Append categorical means to training and scoring x files
        for key, dataset in self.Xs.items():
            # Transaction month
            self.Xs[key]['transaction_month'] = self.Xs[key]['transactiondate'].dt.month
            self.Xs[key].drop('transactiondate', axis=1, inplace=True)
            for column in self.mean_cols:
                self.Xs[key] = pd.merge(self.Xs[key], self.mean_col_dfs_[column], how='left', on=column)
                self.Xs[key].drop(column, axis=1, inplace=True)
                self.Xs[key][column+'_mean_miss'] = self.Xs[key][column+'_mean'].isnull()
                self.Xs[key][column+'_mean'].fillna(0, inplace=True)

In [4]:
drivers = CreateDatasets(path_to_data='/Users/socrteas/Documents/Kaggle/zillow/data/', properties_file='properties_2016.csv', train_file='train_2016_v2.csv')
drivers.run()

In [5]:
class BaseFeatureTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, transforms = [("impute", Imputer()), ("scale", StandardScaler())]):
    def __init__(self):
#         self.transforms = transforms
        
        self.nan_subs = {'airconditioningtypeid': 0, #getdummies
            'architecturalstyletypeid': 0, #getdummies
            'basementsqft': 0,
            'bathroomcnt': 0, 
            'bedroomcnt': 0, 
            'buildingclasstypeid': 0, #get_dummies
            'buildingqualitytypeid': 0,  #get_dummies
            'calculatedbathnbr': 0, 
            'decktypeid': 0, #actually binary
            'finishedfloor1squarefeet': 1348, 
            'calculatedfinishedsquarefeet': 1773,
            'finishedsquarefeet12': 1745, 
            'finishedsquarefeet13': 1404, 
            'finishedsquarefeet15': 2380,
            'finishedsquarefeet50': 1355, 
            'finishedsquarefeet6': 2303, 
            'fips': 0, #getdummies
            'fireplacecnt': 0,
            'fullbathcnt': 0, 
            'garagecarcnt': 0, 
            'garagetotalsqft': 0, 
            'hashottuborspa': False,
            'heatingorsystemtypeid': 0, #getdummies
            'latitude': 0, 
            'longitude': 0, 
            'lat_bins': 0,
            'lon_bins': 0,
            'lotsizesquarefeet': 0, #Lots of heteroscedasticity here, high values have low logerrors
            'poolcnt': 0, 
            'poolsizesum': 0, #Could use 520 (mean) here for rows where poolcnt = 1
            'pooltypeid10': 0, 
            'pooltypeid2': 0, 
            'pooltypeid7': 0,
#             'propertycountylandusecode': '0', 
            'propertylandusetypeid': 0,
#             'propertyzoningdesc': '0', 
#             'rawcensustractandblock': 0, 
#             'regionidcity': 0,
            'regionidcounty': 0, #getdummies
#             'regionidneighborhood': 0, 
#             'regionidzip': 0, 
            'roomcnt': 0,
            'storytypeid': 0, 
            'threequarterbathnbr': 0, 
            'typeconstructiontypeid': 0, #getdummies
            'unitcnt': 1, 
            'yardbuildingsqft17': 0, 
            'yardbuildingsqft26': 0, 
            'yearbuilt': 1968,
            'numberofstories': 1, 
            'fireplaceflag': False, 
            'structuretaxvaluedollarcnt': 180000,  #Lots of heteroscedasticity here, high values have low logerrors
            'taxvaluedollarcnt': 450000,   #Lots of heteroscedasticity here, high values have low logerrors
#             'assessmentyear': 2015,
            'landtaxvaluedollarcnt': 280000, #Lots of heteroscedasticity here, high values have low logerrors
            'taxamount': 6000,   #Lots of heteroscedasticity here, high values have low logerrors
            'taxdelinquencyflag': 'N', 
            'taxdelinquencyyear': 16,
#             'censustractandblock': '0'
           }
        
        self.taxdelinquencyflag_mapping = {'Y': 1, 'N': 0}
        
    def fit(self, X, y=None):
        # Create latitude and longitude bins
        trash1, self.lat_bins_ = pd.cut(X.latitude, bins=100, labels=False, retbins=True)
        trash2, self.lon_bins_ = pd.cut(X.longitude, bins=100, labels=False, retbins=True)
        return self
    
    def transform(self, X, y=None):
#         # Transaction month
#         X['transaction_month'] = X['transactiondate'].dt.month
#         X.drop('transactiondate', axis=1, inplace=True)
        
        # Bin latitude and longitude to avoid overfit
        # Need to create lat_bins and lon_bins as class attributes when the class is created
        X['lat_bins'] = pd.cut(X.latitude, self.lat_bins_, labels=np.arange(100))
        X['lon_bins'] = pd.cut(X.longitude, self.lon_bins_, labels=np.arange(100))
        
        # Create missing indicators
        for key in self.nan_subs:
            X[key+'_miss'] = X[key].isnull()
            
        # Fill NaN's with imputations
        X.fillna(self.nan_subs, inplace=True)
        
        # Convert ordinal characters to ordinal numeric
        X.taxdelinquencyflag = X.taxdelinquencyflag.map(self.taxdelinquencyflag_mapping)
        
#         self.columns_ = X.columns
#         self.feature_columns_ = X.columns.drop(self.drop_cols)
#         self.pipe = Pipeline(self.transforms).fit(X.ix[:, self.feature_columns_])
        
#         cat_cols = X.drop(self.non_cat_columns_.values, 1).reset_index(drop = True)
#         scaled_df = pd.concat([scaled_cols, cat_cols], axis = 1)
#         final_matrix = (pd.get_dummies(scaled_df)
#                         .reindex(columns = self.transformed_columns_)
#                         .fillna(0).as_matrix())
        return X.values()

In [ ]:
# create feature transformation and training pipeline
preprocess = BaseFeatureTransformer()
params = {'subsample': 0.5, 'min_samples_leaf': 100, 'learning_rate': 0.02, 'max_features': 0.3, 'max_depth': 8}
gbm = GradientBoostingRegressor(loss='lad', random_state=90210)
pipe = Pipeline([
    ("preprocess", preprocess), 
    ("gbm", gbm)
])

#                       dict(gbm__n_estimators = [500, 800, 1000, 1200],
#                            gbm__learning_rate = [0.02, 0.01],
#                            gbm__max_depth = [4, 6, 8],
#                            gbm__min_samples_leaf = [50, 100],
#                            gbm__subsample = [0.5],
#                            gbm__max_features = [0.5, 0.3, 0.1])

param_grid = {'n_estimators': [50, 80],
              'learning_rate': [0.02, 0.01],
              'max_depth': [4, 6, 8],
              'min_samples_leaf': [50, 100],
              'subsample': [0.5],
              'max_features': [0.5, 0.3]
              }

# fit model
gbm_cv = GridSearchCV(gbm,
                      param_grid,
#                       cv = 5, # Default is 3-fold CV
                      n_jobs = 4,
                      scoring = make_scorer(mean_absolute_error))

test = BaseFeatureTransformer().fit_transform(drivers.Xs['x_train'])
gbm_cv.fit(test, drivers.y_train)

In [ ]:
print(drivers.Xs['x_train'].columns)
print(drivers.Xs['x_train'].describe())

In [ ]:
gbm_cv.predict(drivers.Xs['x_scoring'])

# Modeling ideas
Get dummies for aforementioned columns with several unique values

Remove outlier logerrors

Test difference on leaderboard when holdout validation isn't good. Does leaderboard position actually suffer?

Include multiplier interactions (create class in pipeline to create)

Stack models